In [52]:
import json
import os
import pandas as pd
from pprint import pprint
from tqdm import tqdm

info needed
-preposition
-category
-image file name
-object 1 info
-object 2 info
-object 1 2d bb coordinates, xtopleft,y topright, width, height
-object 2 2d bb coordinates, xtopleft,y topright, width, height
-object 1 3d bb
-object 2 3d bb

*make it generalisable, set a frames_per_iteration variable,
*some scenes have more than 1 object

# static 2 objects

In [191]:
preposition_name="above"
root_folder_path="D:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
for category in tqdm(categories):
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]

                try:
                    object1_info_dict=info_2d["values"][0]
                    object2_info_dict=info_2d["values"][1]
                    object1_info=object1_info_dict["label_name"]
                    object2_info=object2_info_dict["label_name"]

                    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
                    object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]

                    object1_info_dict=info_3d["values"][0]
                    object2_info_dict=info_3d["values"][1]
                    object1_bb_3d=object1_info_dict["size"]
                    object2_bb_3d=object2_info_dict["size"]

                    new_row={
                        "preposition":preposition_name,
                        "category":category,
                        "image_file_name":filename,
                        "object_1_information":[object1_info],
                        "object_2_information":[object2_info],
                        "object_1_2d_bounding_boxes":[object1_bb_2d],
                        "object_2_2d_bounding_boxes":[object2_bb_2d],
                        "object_1_3d_bounding_boxes":[object1_bb_3d],
                        "object_2_3d_bounding_boxes":[object2_bb_3d],    
                    }
                except:
                    object1_info=[]
                    object2_info=[]
                    object1_bb_2d=[]
                    object2_bb_2d=[]
                    object1_bb_3d=[]
                    object2_bb_3d=[]
#                     print(f"error for {preposition_name}, {category}, {filename}, df index: {len(df)} ")
                    new_row={
                        "preposition":preposition_name,
                        "category":category,
                        "image_file_name":filename,
                        "object_1_information":object1_info,
                        "object_2_information":object2_info
                        "object_1_2d_bounding_boxes":object1_bb_2d,
                        "object_2_2d_bounding_boxes":object2_bb_2d,
                        "object_1_3d_bounding_boxes":object1_bb_3d,
                        "object_2_3d_bounding_boxes":object2_bb_3d,    
                    }


                df.loc[len(df)] = new_row

df

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.51it/s]


,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
1,above,furniture furniture,rgb_3.png,furniture_stove_model_normalized_18 Variant,furniture_table_model_normalized_13 Variant,"[116.0, 214.0, 91.0, 27.0]","[81.0, 408.0, 183.0, 104.0]","{'x': 4.16218758, 'y': 1.2486583, 'z': 4.404978}","{'x': 8.897922, 'y': 3.86387515, 'z': 5.519822}"
2,above,furniture furniture,rgb_4.png,furniture_bed_model_normalized_2 Variant,furniture_microwave_model_normalized_11 Variant,"[129.0, 56.0, 251.0, 224.0]","[188.0, 378.0, 83.0, 48.0]","{'x': 7.91541433, 'y': 10.5797644, 'z': 13.442...","{'x': 4.176357, 'y': 2.172134, 'z': 3.15226078}"
3,above,furniture furniture,rgb_5.png,furniture_table_model_normalized_8 Variant,furniture_stove_model_normalized_22 Variant,"[228.0, 16.0, 167.0, 143.0]","[232.0, 304.0, 72.0, 42.0]","{'x': 10.3031015, 'y': 4.13222027, 'z': 4.4939...","{'x': 4.489478, 'y': 0.908768952, 'z': 3.9690752}"
4,above,furniture furniture,rgb_6.png,furniture_microwave_model_normalized_5 Variant,furniture_microwave_model_normalized_9 Variant,"[173.0, 168.0, 63.0, 44.0]","[109.0, 368.0, 85.0, 60.0]","{'x': 4.947596, 'y': 2.33335924, 'z': 1.622467}","{'x': 4.56189966, 'y': 2.02685046, 'z': 2.9356..."
...,...,...,...,...,...,...,...,...,...
4995,above,vehicles vehicles,rgb_49.png,vehicles_motorcycle_model_normalized_26 Variant,vehicles_motorcycle_model_normalized_18 Variant,"[99.0, 100.0, 79.0, 65.0]","[122.0, 323.0, 136.0, 89.0]","{'x': 3.68977356, 'y': 4.10723257, 'z': 13.912...","{'x': 2.55518985, 'y': 7.201745, 'z': 12.1892366}"
4996,above,vehicles vehicles,rgb_50.png,vehicles_bus_model_normalized_29 Variant,vehicles_bus_model_normalized_22 Variant,"[63.0, 69.0, 418.0, 197.0]","[51.0, 346.0, 416.0, 166.0]","{'x': 10.6660748, 'y': 11.8382, 'z': 40.5224266}","{'x': 9.244875, 'y': 8.296465, 'z': 37.9023476}"
4997,above,vehicles vehicles,rgb_51.png,vehicles_bus_model_normalized_29 Variant,vehicles_bus_model_normalized_29 Variant,"[107.0, 0.0, 405.0, 147.0]","[208.0, 230.0, 304.0, 282.0]","{'x': 10.7983313, 'y': 11.9849939, 'z': 41.02489}","{'x': 10.8080921, 'y': 11.9958267, 'z': 41.061..."
4998,above,vehicles vehicles,rgb_52.png,vehicles_motorcycle_model_normalized_9 Variant,vehicles_motorcycle_model_normalized_21 Variant,"[176.0, 52.0, 76.0, 78.0]","[211.0, 263.0, 93.0, 95.0]","{'x': 4.42329741, 'y': 4.50555849, 'z': 12.064...","{'x': 3.411249, 'y': 5.572359, 'z': 11.4474268}"


In [264]:
o=[]
[o]

[[]]

In [209]:
file=f"{preposition_name}.csv"
print(file)
df.to_csv(file,index=False)

# static multiple objects

In [3]:
import os
import pandas as pd
from tqdm import tqdm

preposition_name="against"
root_folder_path="E:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
for category in tqdm(categories):
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]
                    
                try:
                    object_info_dict=info_2d["values"]
                    sorted_data = sorted(object_info_dict, key=lambda x: x['instance_id'])
                    first_instance_label_id = sorted_data[0]['label_id']
                    list1 = []
                    list2 = []
                    for item in sorted_data:
                        if item['label_id'] == first_instance_label_id:
                            list1.append(item)
                        else:
                            list2.append(item)
                    
                    object1_info=[x["label_name"] for x in list1]
                    object2_info=[x["label_name"] for x in list2]
                    object1_bb_2d=[[x["x"],x["y"],x["width"],x["height"]] for x in list1]
                    object2_bb_2d=[[x["x"],x["y"],x["width"],x["height"]] for x in list2]

                    object_info_dict=info_3d["values"]
                    sorted_data = sorted(object_info_dict, key=lambda x: x['instance_id'])
                    first_instance_label_id = sorted_data[0]['label_id']
                    list1 = []
                    list2 = []
                    for item in sorted_data:
                        if item['label_id'] == first_instance_label_id:
                            list1.append(item)
                        else:
                            list2.append(item)
                    object1_bb_3d=[[x["size"]] for x in list1]
                    object2_bb_3d=[[x["size"]] for x in list2]
                    
                except:
                    object1_info=[]
                    object2_info=[]
                    object1_bb_2d=[]
                    object2_bb_2d=[]
                    object1_bb_3d=[]
                    object2_bb_3d=[]
#                     print(f"error for {preposition_name}, {category}, {filename}, df index: {len(df)} ")

                new_row={
                    "preposition":preposition_name,
                    "category":category,
                    "image_file_name":filename,
                    "object_1_information":object1_info,
                    "object_2_information":object2_info,
                    "object_1_2d_bounding_boxes":object1_bb_2d,
                    "object_2_2d_bounding_boxes":object2_bb_2d,
                    "object_1_3d_bounding_boxes":object1_bb_3d,
                    "object_2_3d_bounding_boxes":object2_bb_3d,    
                }
                df.loc[len(df)] = new_row
    break
df

  0%|                                                                                           | 0/13 [01:09<?, ?it/s]


,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,against,small small small small,rgb_2.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[347.0, 170.0, 43.0, 31.0]]","[[77.0, 247.0, 92.0, 41.0]]","[[{'x': 2.09342766, 'y': 1.3746804, 'z': 1.890...","[[{'x': 2.03364682, 'y': 1.2922132, 'z': 2.034..."
1,against,small small small small,rgb_3.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[343.0, 172.0, 42.0, 31.0]]","[[77.0, 246.0, 94.0, 42.0]]","[[{'x': 2.09342766, 'y': 1.3746804, 'z': 1.890...","[[{'x': 2.03364682, 'y': 1.2922132, 'z': 2.034..."
2,against,small small small small,rgb_4.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[341.0, 173.0, 41.0, 30.0]]","[[77.0, 245.0, 95.0, 44.0]]","[[{'x': 2.09342766, 'y': 1.3746804, 'z': 1.890...","[[{'x': 2.03364682, 'y': 1.2922132, 'z': 2.034..."
3,against,small small small small,rgb_5.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[339.0, 174.0, 41.0, 30.0]]","[[77.0, 245.0, 96.0, 44.0]]","[[{'x': 2.09342766, 'y': 1.3746804, 'z': 1.890...","[[{'x': 2.03364682, 'y': 1.2922132, 'z': 2.034..."
4,against,small small small small,rgb_6.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[335.0, 175.0, 40.0, 30.0]]","[[77.0, 244.0, 98.0, 45.0]]","[[{'x': 2.09342766, 'y': 1.3746804, 'z': 1.890...","[[{'x': 2.03364682, 'y': 1.2922132, 'z': 2.034..."
...,...,...,...,...,...,...,...,...,...
29095,against,small small small small,rgb_29097.png,[small objects_mug_model_normalized_2 Variant],[small objects_bottle_model_normalized_2 Variant],"[[254.0, 432.0, 14.0, 16.0]]","[[228.0, 142.0, 27.0, 25.0]]","[[{'x': 0.6947509, 'y': 0.7600296, 'z': 0.8368...","[[{'x': 1.50328577, 'y': 2.15728474, 'z': 1.50..."
29096,against,small small small small,rgb_29098.png,[small objects_mug_model_normalized_2 Variant],[small objects_bottle_model_normalized_2 Variant],"[[256.0, 436.0, 14.0, 16.0]]","[[226.0, 137.0, 28.0, 25.0]]","[[{'x': 0.6947509, 'y': 0.7600296, 'z': 0.8368...","[[{'x': 1.50328577, 'y': 2.15728474, 'z': 1.50..."
29097,against,small small small small,rgb_29099.png,[small objects_mug_model_normalized_2 Variant],[small objects_bottle_model_normalized_2 Variant],"[[257.0, 439.0, 14.0, 16.0]]","[[225.0, 135.0, 28.0, 24.0]]","[[{'x': 0.6947509, 'y': 0.7600296, 'z': 0.8368...","[[{'x': 1.50328577, 'y': 2.15728474, 'z': 1.50..."
29098,against,small small small small,rgb_29100.png,[small objects_mug_model_normalized_2 Variant],[small objects_bottle_model_normalized_2 Variant],"[[259.0, 444.0, 14.0, 16.0]]","[[223.0, 129.0, 28.0, 25.0]]","[[{'x': 0.6947509, 'y': 0.7600296, 'z': 0.8368...","[[{'x': 1.50328577, 'y': 2.15728474, 'z': 1.50..."


In [259]:
file=f"{preposition_name}.csv"
print(file)
df.to_csv(file,index=False)

all over.csv


# dynamic 2 objects

In [261]:
preposition_name="towards"
root_folder_path="E:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
for category in tqdm(categories):
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]

                try:
                    object1_info_dict=info_2d["values"][0]
                    object2_info_dict=info_2d["values"][1]
                    object1_info=object1_info_dict["label_name"]
                    object2_info=object2_info_dict["label_name"]

                    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
                    object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]

                    object1_info_dict=info_3d["values"][0]
                    object2_info_dict=info_3d["values"][1]
                    object1_bb_3d=object1_info_dict["size"]
                    object2_bb_3d=object2_info_dict["size"]
                    new_row={
                        "preposition":preposition_name,
                        "category":category,
                        "image_file_name":filename,
                        "object_1_information":[object1_info],
                        "object_2_information":[object2_info],
                        "object_1_2d_bounding_boxes":[object1_bb_2d],
                        "object_2_2d_bounding_boxes":[object2_bb_2d],
                        "object_1_3d_bounding_boxes":[object1_bb_3d],
                        "object_2_3d_bounding_boxes":[object2_bb_3d],    
                    }
                except:
                    object1_info=[]
                    object2_info=[]
                    object1_bb_2d=[]
                    object2_bb_2d=[]
                    object1_bb_3d=[]
                    object2_bb_3d=[]
#                     print(f"error for {preposition_name}, {category}, {filename}, df index: {len(df)} ")
                    new_row={
                        "preposition":preposition_name,
                        "category":category,
                        "image_file_name":filename,
                        "object_1_information":object1_info,
                        "object_2_information":object2_info
                        "object_1_2d_bounding_boxes":object1_bb_2d,
                        "object_2_2d_bounding_boxes":object2_bb_2d,
                        "object_1_3d_bounding_boxes":object1_bb_3d,
                        "object_2_3d_bounding_boxes":object2_bb_3d,    
                    }

                df.loc[len(df)] = new_row

df

100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [29:00<00:00, 124.30s/it]


,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,towards,small small small small,rgb_2.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[380.0, 219.0, 50.0, 46.0]]","[[169.0, 243.0, 30.0, 34.0]]","[{'x': 1.19274032, 'y': 0.7832306, 'z': 1.0770...","[{'x': 1.0662595, 'y': 0.677519, 'z': 1.066527..."
1,towards,small small small small,rgb_3.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[370.0, 221.0, 48.0, 45.0]]","[[169.0, 243.0, 30.0, 34.0]]","[{'x': 1.19274032, 'y': 0.7832306, 'z': 1.0770...","[{'x': 1.0662595, 'y': 0.677519, 'z': 1.066527..."
2,towards,small small small small,rgb_4.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[364.0, 222.0, 46.0, 45.0]]","[[169.0, 243.0, 30.0, 34.0]]","[{'x': 1.19274032, 'y': 0.7832306, 'z': 1.0770...","[{'x': 1.0662595, 'y': 0.677519, 'z': 1.066527..."
3,towards,small small small small,rgb_5.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[357.0, 223.0, 46.0, 45.0]]","[[169.0, 243.0, 30.0, 34.0]]","[{'x': 1.19274032, 'y': 0.7832306, 'z': 1.0770...","[{'x': 1.0662595, 'y': 0.677519, 'z': 1.066527..."
4,towards,small small small small,rgb_6.png,[small objects_helmet_model_normalized_3 Variant],[small objects_laptop_model_normalized_2 Variant],"[[349.0, 224.0, 43.0, 45.0]]","[[169.0, 243.0, 30.0, 34.0]]","[{'x': 1.19274032, 'y': 0.7832306, 'z': 1.0770...","[{'x': 1.0662595, 'y': 0.677519, 'z': 1.066527..."
...,...,...,...,...,...,...,...,...,...
149995,towards,person building,rgb_4437.png,[person_person_1 Variant],[grounded_large_objects_train_model_normalized...,"[[273.0, 309.0, 45.0, 54.0]]","[[235.0, 136.0, 239.0, 229.0]]","[{'x': 656.709, 'y': 2.762651, 'z': 2300.71948}]","[{'x': 3.43024063, 'y': 4.78804, 'z': 10.34081}]"
149996,towards,person building,rgb_4438.png,[grounded_large_objects_train_model_normalized...,[person_person_1 Variant],"[[235.0, 136.0, 239.0, 229.0]]","[[279.0, 312.0, 42.0, 53.0]]","[{'x': 3.43024063, 'y': 4.78804, 'z': 10.34081}]","[{'x': 656.709, 'y': 2.762651, 'z': 2300.71948}]"
149997,towards,person building,rgb_4439.png,[grounded_large_objects_train_model_normalized...,[person_person_1 Variant],"[[235.0, 136.0, 239.0, 229.0]]","[[288.0, 316.0, 40.0, 51.0]]","[{'x': 3.43024063, 'y': 4.78804, 'z': 10.34081}]","[{'x': 656.709, 'y': 2.762651, 'z': 2300.71948}]"
149998,towards,person building,rgb_4440.png,[grounded_large_objects_train_model_normalized...,[person_person_1 Variant],"[[235.0, 136.0, 239.0, 229.0]]","[[293.0, 313.0, 40.0, 55.0]]","[{'x': 3.43024063, 'y': 4.78804, 'z': 10.34081}]","[{'x': 656.709, 'y': 2.762651, 'z': 2300.71948}]"


In [262]:
file=f"{preposition_name}.csv"
print(file)
df.to_csv(file,index=False)

towards.csv


# dynamic 1 object

In [215]:
preposition_name="down"
root_folder_path="E:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_1_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
for category in tqdm(categories):
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]

                try:
                    object1_info_dict=info_2d["values"][0]
                    object1_info=object1_info_dict["label_name"]

                    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]

                    object1_info_dict=info_3d["values"][0]
                    object1_bb_3d=object1_info_dict["size"]
                    object2_info=[]
                    object2_bb_2d=[]
                    object2_bb_3d=[]
                    new_row={
                        "preposition":preposition_name,
                        "category":category,
                        "image_file_name":filename,
                        "object_1_information":[object1_info],
                        "object_2_information":object2_info,
                        "object_1_2d_bounding_boxes":[object1_bb_2d],
                        "object_2_2d_bounding_boxes":object2_bb_2d,
                        "object_1_3d_bounding_boxes":[object1_bb_3d],
                        "object_2_3d_bounding_boxes":object2_bb_3d,    
                    }
                    
                except:
                    object1_info=[]
                    object1_bb_2d=[]
                    object1_bb_3d=[]
                    object2_info=[]
                    object2_bb_2d=[]
                    object2_bb_3d=[]
#                     print(f"error for {preposition_name}, {category}, {filename}, df index: {len(df)} ")
                    new_row={
                        "preposition":preposition_name,
                        "category":category,
                        "image_file_name":filename,
                        "object_1_information":object1_info,
                        "object_2_information":object2_info,
                        "object_1_2d_bounding_boxes":object1_bb_2d,
                        "object_2_2d_bounding_boxes":object2_bb_2d,
                        "object_1_3d_bounding_boxes":object1_bb_3d,
                        "object_2_3d_bounding_boxes":object2_bb_3d,    
                    }
                    

                df.loc[len(df)] = new_row

df

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [19:05<00:00, 229.08s/it]


,preposition,category,image_file_name,object_1_information,object_1_2d_bounding_boxes,object_1_3d_bounding_boxes
0,down,furniture,rgb_2.png,furniture_sofa_model_normalized_28 Variant,"[55.0, 217.0, 156.0, 181.0]","{'x': 9.379209, 'y': 4.473857, 'z': 4.56427956}"
1,down,furniture,rgb_3.png,furniture_sofa_model_normalized_28 Variant,"[56.0, 222.0, 155.0, 185.0]","{'x': 9.379209, 'y': 4.473857, 'z': 4.56427956}"
2,down,furniture,rgb_4.png,furniture_sofa_model_normalized_28 Variant,"[56.0, 229.0, 156.0, 189.0]","{'x': 9.379209, 'y': 4.473857, 'z': 4.56427956}"
3,down,furniture,rgb_5.png,furniture_sofa_model_normalized_28 Variant,"[56.0, 235.0, 156.0, 192.0]","{'x': 9.379209, 'y': 4.473857, 'z': 4.56427956}"
4,down,furniture,rgb_6.png,furniture_sofa_model_normalized_28 Variant,"[57.0, 241.0, 156.0, 197.0]","{'x': 9.379209, 'y': 4.473857, 'z': 4.56427956}"
...,...,...,...,...,...,...
149995,down,vehicles,rgb_13617.png,vehicles_motorcycle_model_normalized_11 Variant,"[128.0, 322.0, 109.0, 63.0]","{'x': 3.44854832, 'y': 3.535281, 'z': 6.99509}"
149996,down,vehicles,rgb_13618.png,vehicles_motorcycle_model_normalized_11 Variant,"[128.0, 326.0, 109.0, 63.0]","{'x': 3.44854832, 'y': 3.535281, 'z': 6.99509}"
149997,down,vehicles,rgb_13619.png,vehicles_motorcycle_model_normalized_11 Variant,"[127.0, 332.0, 109.0, 64.0]","{'x': 3.44854832, 'y': 3.535281, 'z': 6.99509}"
149998,down,vehicles,rgb_13620.png,vehicles_motorcycle_model_normalized_11 Variant,"[127.0, 337.0, 108.0, 63.0]","{'x': 3.44854832, 'y': 3.535281, 'z': 6.99509}"


In [217]:
file=f"{preposition_name}.csv"
print(file)
df.to_csv(file,index=False)

# applies to all except up and down

In [7]:
import os
import pandas as pd
from tqdm import tqdm

preposition_name="below"
root_folder_path="E:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
for category in tqdm(categories):
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]
                    
                try:
                    object_info_dict=info_2d["values"]
                    sorted_data = sorted(object_info_dict, key=lambda x: x['instance_id'])
                    first_instance_label_id = sorted_data[0]['label_id']
                    list1 = []
                    list2 = []
                    for item in sorted_data:
                        if item['label_id'] == first_instance_label_id:
                            list1.append(item)
                        else:
                            list2.append(item)
                    object1_info=[x["label_name"] for x in list1]
                    object2_info=[x["label_name"] for x in list2]
                    object1_bb_2d=[[x["x"],x["y"],x["width"],x["height"]] for x in list1]
                    object2_bb_2d=[[x["x"],x["y"],x["width"],x["height"]] for x in list2]

                    object_info_dict=info_3d["values"]
                    sorted_data = sorted(object_info_dict, key=lambda x: x['instance_id'])
                    first_instance_label_id = sorted_data[0]['label_id']
                    list1 = []
                    list2 = []
                    for item in sorted_data:
                        if item['label_id'] == first_instance_label_id:
                            list1.append(item)
                        else:
                            list2.append(item)
                    object1_bb_3d=[[x["size"]] for x in list1]
                    object2_bb_3d=[[x["size"]] for x in list2]
                    
                except:
                    object1_info=[]
                    object2_info=[]
                    object1_bb_2d=[]
                    object2_bb_2d=[]
                    object1_bb_3d=[]
                    object2_bb_3d=[]
#                     print(f"error for {preposition_name}, {category}, {filename}, df index: {len(df)} ")

                new_row={
                    "preposition":preposition_name,
                    "category":category,
                    "image_file_name":filename,
                    "object_1_information":object1_info,
                    "object_2_information":object2_info,
                    "object_1_2d_bounding_boxes":object1_bb_2d,
                    "object_2_2d_bounding_boxes":object2_bb_2d,
                    "object_1_3d_bounding_boxes":object1_bb_3d,
                    "object_2_3d_bounding_boxes":object2_bb_3d,    
                }
                df.loc[len(df)] = new_row
        
        
print(preposition_name)
print(len(df))
df.tail()

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:06<00:00,  2.16it/s]

below
5000


,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
4995,below,vehicles vehicles,rgb_49.png,[vehicles_motorcycle_model_normalized_26 Variant],[vehicles_motorcycle_model_normalized_18 Variant],"[[210.0, 365.0, 46.0, 89.0]]","[[130.0, 73.0, 131.0, 96.0]]","[[{'x': 3.68977356, 'y': 4.10723257, 'z': 13.9...","[[{'x': 2.55518985, 'y': 7.20174551, 'z': 12.1..."
4996,below,vehicles vehicles,rgb_50.png,[vehicles_bus_model_normalized_29 Variant],[vehicles_bus_model_normalized_22 Variant],"[[41.0, 247.0, 431.0, 265.0]]","[[132.0, 108.0, 380.0, 127.0]]","[[{'x': 10.6660748, 'y': 11.8382, 'z': 40.5224...","[[{'x': 9.244875, 'y': 8.296465, 'z': 37.90234}]]"
4997,below,vehicles vehicles,rgb_51.png,"[vehicles_bus_model_normalized_29 Variant, veh...",[],"[[123.0, 235.0, 389.0, 222.0], [218.0, 0.0, 29...",[],"[[{'x': 10.7983313, 'y': 11.9849939, 'z': 41.0...",[]
4998,below,vehicles vehicles,rgb_52.png,[vehicles_motorcycle_model_normalized_9 Variant],[vehicles_motorcycle_model_normalized_21 Variant],"[[253.0, 278.0, 61.0, 46.0]]","[[200.0, 52.0, 99.0, 88.0]]","[[{'x': 4.42329741, 'y': 4.50555849, 'z': 12.0...","[[{'x': 3.41124964, 'y': 5.57235861, 'z': 11.4..."
4999,below,vehicles vehicles,rgb_53.png,[vehicles_car_model_normalized_7 Variant],[vehicles_motorcycle_model_normalized_26 Variant],"[[155.0, 408.0, 123.0, 104.0]]","[[104.0, 189.0, 122.0, 55.0]]","[[{'x': 8.064554, 'y': 5.40387154, 'z': 18.383...","[[{'x': 3.185494, 'y': 3.54589963, 'z': 12.011..."


In [6]:
file=f"{preposition_name}.csv"
print(file)
df.to_csv(file,index=False)

above.csv


# draft work

In [246]:
object1_bb_3d

[[{'x': 0.469766021, 'y': 0.107374996, 'z': 0.13338}],
 [{'x': 4.15782, 'y': 0.950358033, 'z': 1.180524}],
 [{'x': 4.53435564, 'y': 1.03642273, 'z': 1.28743279}]]

In [244]:
list1

[{'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 1,
  'translation': {'x': 3.19822, 'y': -1.02900708, 'z': 18.688015},
  'size': {'x': 0.469766021, 'y': 0.107374996, 'z': 0.13338},
  'rotation': {'x': 0.05635103,
   'y': -0.9942864,
   'z': -0.0377428681,
   'w': 0.08242932},
  'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
  'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 2,
  'translation': {'x': 8.362261, 'y': -3.56315351, 'z': 16.6246834},
  'size': {'x': 4.15782, 'y': 0.950358033, 'z': 1.180524},
  'rotation': {'x': -0.459470361,
   'y': -0.5394858,
   'z': -0.205046311,
   'w': 0.6751283},
  'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
  'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 3,
  'translation': {'x': 2.83957267, 'y': -4.389583, 'z'

In [242]:
sorted_data

[{'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 1,
  'translation': {'x': 3.19822, 'y': -1.02900708, 'z': 18.688015},
  'size': {'x': 0.469766021, 'y': 0.107374996, 'z': 0.13338},
  'rotation': {'x': 0.05635103,
   'y': -0.9942864,
   'z': -0.0377428681,
   'w': 0.08242932},
  'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
  'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 2,
  'translation': {'x': 8.362261, 'y': -3.56315351, 'z': 16.6246834},
  'size': {'x': 4.15782, 'y': 0.950358033, 'z': 1.180524},
  'rotation': {'x': -0.459470361,
   'y': -0.5394858,
   'z': -0.205046311,
   'w': 0.6751283},
  'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
  'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 3,
  'translation': {'x': 2.83957267, 'y': -4.389583, 'z'

In [240]:
info_3d

{'id': 'bounding box 3D',
 'annotation_definition': 'bounding box 3D',
 'values': [{'label_id': 36,
   'label_name': 'furniture_cabinet_model_normalized_23 Variant',
   'instance_id': 7,
   'translation': {'x': 2.82809258, 'y': 4.5293, 'z': 20.3237171},
   'size': {'x': 2.05600452, 'y': 3.40815067, 'z': 0.9668772},
   'rotation': {'x': -0.8391439,
    'y': 0.0279905386,
    'z': 0.125597835,
    'w': -0.5284687},
   'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
   'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
  {'label_id': 36,
   'label_name': 'furniture_cabinet_model_normalized_23 Variant',
   'instance_id': 8,
   'translation': {'x': 1.67077541, 'y': 0.635308743, 'z': 15.8595285},
   'size': {'x': 1.79432666, 'y': 2.974378, 'z': 0.8438178},
   'rotation': {'x': 0.3146055,
    'y': -0.608799,
    'z': -0.6324266,
    'w': -0.361142367},
   'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
   'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
  {'label_id': 36,
   'label_name': 'furniture_ca

In [236]:
object2_info

['furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant']

In [237]:
object1_bb_2d

[[340.0, 283.0, 13.0, 3.0],
 [494.0, 334.0, 18.0, 53.0],
 [276.0, 377.0, 143.0, 64.0]]

In [238]:
object2_bb_2d

[[342.0, 276.0, 8.0, 12.0],
 [376.0, 229.0, 51.0, 50.0],
 [381.0, 269.0, 73.0, 71.0],
 [300.0, 94.0, 62.0, 84.0],
 [253.0, 215.0, 117.0, 35.0],
 [406.0, 367.0, 50.0, 116.0],
 [277.0, 214.0, 81.0, 120.0]]

In [233]:
sorted_data

[{'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 1,
  'x': 340.0,
  'y': 283.0,
  'width': 13.0,
  'height': 3.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 2,
  'x': 494.0,
  'y': 334.0,
  'width': 18.0,
  'height': 53.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 3,
  'x': 276.0,
  'y': 377.0,
  'width': 143.0,
  'height': 64.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 4,
  'x': 342.0,
  'y': 276.0,
  'width': 8.0,
  'height': 12.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 5,
  'x': 376.0,
  'y': 229.0,
  'width': 51.0,
  'height': 50.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 6,
  'x': 381.0,
  'y': 269.0,
  'width': 73.0,
  'height': 71.0},
 {'la

In [231]:
object1_info

['furniture_bench_model_normalized_26 Variant',
 'furniture_bench_model_normalized_26 Variant',
 'furniture_bench_model_normalized_26 Variant']

In [232]:
object2_info

['furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant',
 'furniture_cabinet_model_normalized_23 Variant']

In [227]:
list1

[{'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 1,
  'x': 340.0,
  'y': 283.0,
  'width': 13.0,
  'height': 3.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 2,
  'x': 494.0,
  'y': 334.0,
  'width': 18.0,
  'height': 53.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 3,
  'x': 276.0,
  'y': 377.0,
  'width': 143.0,
  'height': 64.0}]

In [229]:
list2

[{'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 4,
  'x': 342.0,
  'y': 276.0,
  'width': 8.0,
  'height': 12.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 5,
  'x': 376.0,
  'y': 229.0,
  'width': 51.0,
  'height': 50.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 6,
  'x': 381.0,
  'y': 269.0,
  'width': 73.0,
  'height': 71.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 7,
  'x': 300.0,
  'y': 94.0,
  'width': 62.0,
  'height': 84.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 8,
  'x': 253.0,
  'y': 215.0,
  'width': 117.0,
  'height': 35.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 9,
  'x': 406.0,
  'y': 367.0,
  'width': 50.0,
  'height': 116.0}

In [228]:
object1_info_dict=info_2d["values"]
object1_info_dict

[{'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 7,
  'x': 300.0,
  'y': 94.0,
  'width': 62.0,
  'height': 84.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 8,
  'x': 253.0,
  'y': 215.0,
  'width': 117.0,
  'height': 35.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 5,
  'x': 376.0,
  'y': 229.0,
  'width': 51.0,
  'height': 50.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 1,
  'x': 340.0,
  'y': 283.0,
  'width': 13.0,
  'height': 3.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 4,
  'x': 342.0,
  'y': 276.0,
  'width': 8.0,
  'height': 12.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 10,
  'x': 277.0,
  'y': 214.0,
  'width': 81.0,
  'height': 120.0},


In [220]:
sorted_data = sorted(object1_info_dict, key=lambda x: x['instance_id'])
sorted_data

[{'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 1,
  'x': 340.0,
  'y': 283.0,
  'width': 13.0,
  'height': 3.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 2,
  'x': 494.0,
  'y': 334.0,
  'width': 18.0,
  'height': 53.0},
 {'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'instance_id': 3,
  'x': 276.0,
  'y': 377.0,
  'width': 143.0,
  'height': 64.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 4,
  'x': 342.0,
  'y': 276.0,
  'width': 8.0,
  'height': 12.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 5,
  'x': 376.0,
  'y': 229.0,
  'width': 51.0,
  'height': 50.0},
 {'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'instance_id': 6,
  'x': 381.0,
  'y': 269.0,
  'width': 73.0,
  'height': 71.0},
 {'la

In [224]:
from pprint import pprint
sorted_data = sorted(object1_info_dict, key=lambda x: x['instance_id'])

first_instance_label_id = sorted_data[0]['label_id']

list1 = []
list2 = []

for item in sorted_data:
    if item['label_id'] == first_instance_label_id:
        list1.append(item)
    else:
        list2.append(item)
        
pprint(list1)
print("\n")
pprint(list2)

[{'height': 3.0,
  'instance_id': 1,
  'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'width': 13.0,
  'x': 340.0,
  'y': 283.0},
 {'height': 53.0,
  'instance_id': 2,
  'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'width': 18.0,
  'x': 494.0,
  'y': 334.0},
 {'height': 64.0,
  'instance_id': 3,
  'label_id': 29,
  'label_name': 'furniture_bench_model_normalized_26 Variant',
  'width': 143.0,
  'x': 276.0,
  'y': 377.0}]


[{'height': 12.0,
  'instance_id': 4,
  'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'width': 8.0,
  'x': 342.0,
  'y': 276.0},
 {'height': 50.0,
  'instance_id': 5,
  'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'width': 51.0,
  'x': 376.0,
  'y': 229.0},
 {'height': 71.0,
  'instance_id': 6,
  'label_id': 36,
  'label_name': 'furniture_cabinet_model_normalized_23 Variant',
  'width': 73.0,
  'x': 381.0,
  'y': 269.0},
 {'

In [173]:
# test with 1 static example - 2 objects only - above
preposition_name="above"
root_folder_path="D:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)

for category in categories:
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
            #     pprint(data)
            #     pprint(data.keys()) # only need filename and annotations
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
            #     pprint(annotations)
#                 info_3d=annotations[0]
#                 info_2d=annotations[1]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]
            #     pprint(info_2d)
            #     pprint(info_3d)
                object1_info_dict=info_2d["values"][0]
                object2_info_dict=info_2d["values"][1]
                object1_info=object1_info_dict["label_name"]
                object2_info=object2_info_dict["label_name"]
            #     pprint(object1_info)
            #     pprint(object2_info)
                object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
                object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]
            #     pprint(filename)
            #     pprint(object1_bb_2d)
            #     pprint(object2_bb_2d)
                object1_info_dict=info_3d["values"][0]
                object2_info_dict=info_3d["values"][1]
                object1_bb_3d=object1_info_dict["size"]
                object2_bb_3d=object2_info_dict["size"]
                new_row={
                    "preposition":preposition_name,
                    "category":category,
                    "image_file_name":filename,
                    "object_1_information":object1_info,
                    "object_2_information":object2_info,
                    "object_1_2d_bounding_boxes":object1_bb_2d,
                    "object_2_2d_bounding_boxes":object2_bb_2d,
                    "object_1_3d_bounding_boxes":object1_bb_3d,
                    "object_2_3d_bounding_boxes":object2_bb_3d,    
                }
                df.loc[len(df)] = new_row
            #     pprint(new_row)
#                 print(df)
                
        
#         break
#     break

df

IndexError: list index out of range

In [91]:
full_json_path

'D:/SPUR NEURIPS\\above\\furniture furniture\\Datasetccdec446-3a83-4816-84a0-2e53a1e42454\\captures_000.json'

In [92]:
import json

# Open the JSON file
with open(full_json_path) as file:
    json_data = json.load(file)

# Access the JSON data
json_data.keys()

dict_keys(['version', 'captures'])

In [117]:
for data in json_data["captures"]:
#     pprint(data)
#     pprint(data.keys()) # only need filename and annotations
    filename=data["filename"].split("/")[-1]
    annotations=data["annotations"]
#     pprint(annotations)
    info_3d=annotations[0]
    info_2d=annotations[1]
#     pprint(info_2d)
#     pprint(info_3d)
    object1_info_dict=info_2d["values"][0]
    object2_info_dict=info_2d["values"][1]
    object1_info=object1_info_dict["label_name"]
    object2_info=object2_info_dict["label_name"]
#     pprint(object1_info)
#     pprint(object2_info)
    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
    object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]
#     pprint(filename)
#     pprint(object1_bb_2d)
#     pprint(object2_bb_2d)
    object1_info_dict=info_3d["values"][0]
    object2_info_dict=info_3d["values"][1]
    object1_bb_3d=object1_info_dict["size"]
    object2_bb_3d=object2_info_dict["size"]
    new_row={
        "preposition":preposition_name,
        "category":category,
        "image_file_name":filename,
        "object_1_information":object1_info,
        "object_2_information":object2_info,
        "object_1_2d_bounding_boxes":object1_bb_2d,
        "object_2_2d_bounding_boxes":object2_bb_2d,
        "object_1_3d_bounding_boxes":object1_bb_3d,
        "object_2_3d_bounding_boxes":object2_bb_3d,    
    }
    df.loc[len(df)] = new_row
    break
    
df

,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
1,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
2,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
3,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
4,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
5,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
6,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"


In [127]:
#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]

df=pd.DataFrame(columns=columns)

for data in json_data["captures"]:
#     pprint(data)
#     pprint(data.keys()) # only need filename and annotations
    filename=data["filename"].split("/")[-1]
    annotations=data["annotations"]
#     pprint(annotations)
#     info_3d=annotations[0]
#     info_2d=annotations[1]
    if annotations[0]["id"]=="bounding box 3D":
        info_3d=annotations[0]
        info_2d=annotations[1]
    else:
        info_3d=annotations[1]
        info_2d=annotations[0]
#     pprint(info_2d)
#     pprint(info_3d)
    object1_info_dict=info_2d["values"][0]
    object2_info_dict=info_2d["values"][1]
    object1_info=object1_info_dict["label_name"]
    object2_info=object2_info_dict["label_name"]
#     pprint(object1_info)
#     pprint(object2_info)
    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
    object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]
#     pprint(filename)
#     pprint(object1_bb_2d)
#     pprint(object2_bb_2d)
    object1_info_dict=info_3d["values"][0]
    object2_info_dict=info_3d["values"][1]
    object1_bb_3d=object1_info_dict["size"]
    object2_bb_3d=object2_info_dict["size"]
    new_row={
        "preposition":preposition_name,
        "category":category,
        "image_file_name":filename,
        "object_1_information":object1_info,
        "object_2_information":object2_info,
        "object_1_2d_bounding_boxes":object1_bb_2d,
        "object_2_2d_bounding_boxes":object2_bb_2d,
        "object_1_3d_bounding_boxes":object1_bb_3d,
        "object_2_3d_bounding_boxes":object2_bb_3d,    
    }
    df.loc[len(df)] = new_row
#     pprint(new_row)
    
    
    
df

,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"
1,above,furniture furniture,rgb_3.png,furniture_stove_model_normalized_18 Variant,furniture_table_model_normalized_13 Variant,"[116.0, 214.0, 91.0, 27.0]","[81.0, 408.0, 183.0, 104.0]","{'x': 4.16218758, 'y': 1.2486583, 'z': 4.404978}","{'x': 8.897922, 'y': 3.86387515, 'z': 5.519822}"
2,above,furniture furniture,rgb_4.png,furniture_bed_model_normalized_2 Variant,furniture_microwave_model_normalized_11 Variant,"[129.0, 56.0, 251.0, 224.0]","[188.0, 378.0, 83.0, 48.0]","{'x': 7.91541433, 'y': 10.5797644, 'z': 13.442...","{'x': 4.176357, 'y': 2.172134, 'z': 3.15226078}"
3,above,furniture furniture,rgb_5.png,furniture_table_model_normalized_8 Variant,furniture_stove_model_normalized_22 Variant,"[228.0, 16.0, 167.0, 143.0]","[232.0, 304.0, 72.0, 42.0]","{'x': 10.3031015, 'y': 4.13222027, 'z': 4.4939...","{'x': 4.489478, 'y': 0.908768952, 'z': 3.9690752}"
4,above,furniture furniture,rgb_6.png,furniture_microwave_model_normalized_5 Variant,furniture_microwave_model_normalized_9 Variant,"[173.0, 168.0, 63.0, 44.0]","[109.0, 368.0, 85.0, 60.0]","{'x': 4.947596, 'y': 2.33335924, 'z': 1.622467}","{'x': 4.56189966, 'y': 2.02685046, 'z': 2.9356..."
...,...,...,...,...,...,...,...,...,...
146,above,furniture furniture,rgb_148.png,furniture_bench_model_normalized_2 Variant,furniture_bookshelf_model_normalized_10 Variant,"[229.0, 153.0, 102.0, 110.0]","[221.0, 349.0, 109.0, 145.0]","{'x': 12.1412067, 'y': 2.96127415, 'z': 2.90204}","{'x': 10.4835806, 'y': 4.899369, 'z': 2.5287025}"
147,above,furniture furniture,rgb_149.png,furniture_bench_model_normalized_0 Variant,furniture_cabinet_model_normalized_23 Variant,"[172.0, 165.0, 104.0, 90.0]","[118.0, 285.0, 172.0, 227.0]","{'x': 11.4579268, 'y': 4.979709, 'z': 3.58765459}","{'x': 7.401849, 'y': 12.2697268, 'z': 3.48086643}"
148,above,furniture furniture,rgb_150.png,furniture_cabinet_model_normalized_14 Variant,furniture_sofa_model_normalized_28 Variant,"[288.0, 0.0, 130.0, 125.0]","[340.0, 231.0, 172.0, 93.0]","{'x': 7.08201933, 'y': 10.6230211, 'z': 4.2308...","{'x': 9.399388, 'y': 4.483482, 'z': 4.5741}"
149,above,furniture furniture,rgb_151.png,furniture_chair_model_normalized_5 Variant,furniture_sofa_model_normalized_28 Variant,"[302.0, 0.0, 87.0, 115.0]","[254.0, 256.0, 145.0, 86.0]","{'x': 3.40774727, 'y': 5.53674555, 'z': 3.9335...","{'x': 9.16115952, 'y': 4.369847, 'z': 4.458169}"


In [125]:
#     info_3d=annotations[0]
#     info_2d=annotations[1]
# #     
#     annotations=data["annotations"]
if annotations[0]["id"]=="bounding box 3D":
    info_3d=annotations[0]
    info_2d=annotations[1]
else:
    info_3d=annotations[1]
    info_2d=annotations[0]
    

[{'id': 'bounding box 3D',
  'annotation_definition': 'bounding box 3D',
  'values': [{'label_id': 47,
    'label_name': 'furniture_microwave_model_normalized_9 Variant',
    'instance_id': 1,
    'translation': {'x': -3.26375246, 'y': 6.27499866, 'z': 28.3797226},
    'size': {'x': 4.70499229, 'y': 2.09042645, 'z': 3.02772069},
    'rotation': {'x': -0.123021834,
     'y': 0.7578262,
     'z': 0.06434152,
     'w': 0.637514949},
    'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
    'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}},
   {'label_id': 49,
    'label_name': 'furniture_microwave_model_normalized_28 Variant',
    'instance_id': 2,
    'translation': {'x': -3.23442531, 'y': -7.79885864, 'z': 25.3278942},
    'size': {'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044},
    'rotation': {'x': 0.12423858,
     'y': -0.8820308,
     'z': -0.07098353,
     'w': 0.448940784},
    'velocity': {'x': 0.0, 'y': 0.0, 'z': 0.0},
    'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0}}]},
 {'id': 'bou

In [ ]:
for data in json_data["captures"]:


In [102]:
len(json_data["captures"])

151

In [103]:
json_data["captures"][0]

{'id': 'e98b016b-0f99-4ac8-be57-e1b8965c9357',
 'sequence_id': '9a01a5a0-697f-43d5-a863-77179d1f1e46',
 'step': 0,
 'timestamp': 0.0,
 'sensor': {'sensor_id': 'camera',
  'ego_id': 'ego',
  'modality': 'camera',
  'translation': [0.146436423, -0.0978947, 27.86686],
  'rotation': [-0.04609865, 0.9966391, 0.0254077967, -0.06276863],
  'camera_intrinsic': [[1.51409245, 0.0, 0.0],
   [0.0, 1.51409245, 0.0],
   [0.0, 0.0, -1.0006001]],
  'projection': 'perspective'},
 'ego': {'ego_id': 'ego',
  'translation': [0.0, 0.0, 0.0],
  'rotation': [0.0, 0.0, 0.0, 1.0],
  'velocity': None,
  'acceleration': None},
 'filename': 'RGB595ec9b3-a3d7-4366-804a-9775c5695693/rgb_2.png',
 'format': 'PNG',
 'annotations': [{'id': 'bounding box 3D',
   'annotation_definition': 'bounding box 3D',
   'values': [{'label_id': 47,
     'label_name': 'furniture_microwave_model_normalized_9 Variant',
     'instance_id': 1,
     'translation': {'x': -3.26375246, 'y': 6.27499866, 'z': 28.3797226},
     'size': {'x': 4.

In [104]:
json_data["captures"][1]

{'id': 'e7b41f69-fe30-4c10-aa07-bb6b31dbb33c',
 'sequence_id': '9b01a5a0-697f-43d5-a863-77179d1f1e46',
 'step': 0,
 'timestamp': 0.0,
 'sensor': {'sensor_id': 'camera',
  'ego_id': 'ego',
  'modality': 'camera',
  'translation': [0.352349967, 0.3809062, 27.5484047],
  'rotation': [-0.01487619, 0.991568565, 0.07431385, -0.1051091],
  'camera_intrinsic': [[1.72022474, 0.0, 0.0],
   [0.0, 1.72022474, 0.0],
   [0.0, 0.0, -1.0006001]],
  'projection': 'perspective'},
 'ego': {'ego_id': 'ego',
  'translation': [0.0, 0.0, 0.0],
  'rotation': [0.0, 0.0, 0.0, 1.0],
  'velocity': None,
  'acceleration': None},
 'filename': 'RGB595ec9b3-a3d7-4366-804a-9775c5695693/rgb_3.png',
 'format': 'PNG',
 'annotations': [{'id': 'bounding box',
   'annotation_definition': 'bounding box',
   'values': [{'label_id': 58,
     'label_name': 'furniture_stove_model_normalized_18 Variant',
     'instance_id': 1,
     'x': 116.0,
     'y': 214.0,
     'width': 91.0,
     'height': 27.0},
    {'label_id': 65,
     'l

In [96]:
df

,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,above,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_28 Variant,"[182.0, 144.0, 59.0, 47.0]","[173.0, 349.0, 68.0, 52.0]","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277...","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}"


In [85]:
x='captures_000.json'
x.startswith("captures")

True

In [39]:
import json

# Open the JSON file
with open(sample_json_path) as file:
    json_data = json.load(file)

# Access the JSON data
json_data.keys()

dict_keys(['version', 'captures'])

In [40]:
len(json_data["captures"])

151

In [80]:
category=categories[0]
category

'furniture furniture'

In [170]:
for data in json_data["captures"]:
#     pprint(data)
#     pprint(data.keys()) # only need filename and annotations
    filename=data["filename"].split("/")[-1]
    annotations=data["annotations"]
#     pprint(annotations)
    info_3d=annotations[0]
    info_2d=annotations[1]
#     pprint(info_2d)
#     pprint(info_3d)
    object1_info_dict=info_2d["values"][0]
    object2_info_dict=info_2d["values"][1]
    object1_info=object1_info_dict["label_name"]
    object2_info=object2_info_dict["label_name"]
#     pprint(object1_info)
#     pprint(object2_info)
    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
    object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]
#     pprint(filename)
#     pprint(object1_bb_2d)
#     pprint(object2_bb_2d)
    object1_info_dict=info_3d["values"][0]
    object2_info_dict=info_3d["values"][1]
    object1_bb_3d=object1_info_dict["size"]
    object2_bb_3d=object2_info_dict["size"]
    new_row={
        "preposition":preposition_name,
        "category":category,
        "image_file_name":filename,
        "object_1_information":object1_info,
        "object_2_information":object2_info,
        "object_1_2d_bounding_boxes":object1_bb_2d,
        "object_2_2d_bounding_boxes":object2_bb_2d,
        "object_1_3d_bounding_boxes":object1_bb_3d,
        "object_2_3d_bounding_boxes":object2_bb_3d,    
    }
    df=df.append(new_row)
#     pprint(new_row)
    
    break

AttributeError: 'DataFrame' object has no attribute 'append'

In [48]:
for data in json_data["captures"]:
    pprint(data)
#     pprint(data.keys())
    break

{'annotations': [{'annotation_definition': 'bounding box 3D',
                  'id': 'bounding box 3D',
                  'values': [{'acceleration': {'x': 0.0, 'y': 0.0, 'z': 0.0},
                              'instance_id': 1,
                              'label_id': 47,
                              'label_name': 'furniture_microwave_model_normalized_9 '
                                            'Variant',
                              'rotation': {'w': 0.637514949,
                                           'x': -0.123021834,
                                           'y': 0.7578262,
                                           'z': 0.06434152},
                              'size': {'x': 4.70499229,
                                       'y': 2.09042645,
                                       'z': 3.02772069},
                              'translation': {'x': -3.26375246,
                                              'y': 6.27499866,
                                           

In [ ]:
# test with 1 dynamic example - 2 objects only - up
folder_path=""

In [ ]:
# test with 1 static example - >2 objects - among
folder_path=""

In [135]:
categories

['furniture furniture',
 'furniture person',
 'furniture vehicles',
 'person furniture',
 'person person',
 'person small big',
 'person vehicles',
 'small big furniture',
 'small big person',
 'small big small big',
 'small small small small',
 'vehicles furniture',
 'vehicles person',
 'vehicles vehicles']

In [136]:
category=categories[0]

In [172]:
# test with 1 static example - 2 objects only - above
preposition_name="above"
root_folder_path="D:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]

df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
category=categories[8]

category_folder_path=os.path.join(preposition_folder_path,category)
category_folder_path_list=os.listdir(category_folder_path)
category_folder_path_list=sorted(category_folder_path_list)
dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
dataset_folder_path_list=os.listdir(dataset_folder_path)
captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
for captures_json_file in captures_list:
    full_json_path=os.path.join(dataset_folder_path,captures_json_file)
    with open(full_json_path) as file:
        json_data = json.load(file)
        for data in json_data["captures"]:
        #     pprint(data)
        #     pprint(data.keys()) # only need filename and annotations
            filename=data["filename"].split("/")[-1]
            annotations=data["annotations"]
#             pprint(annotations)
            if annotations[0]["id"]=="bounding box 3D":
                info_3d=annotations[0]
                info_2d=annotations[1]
            else:
                info_3d=annotations[1]
                info_2d=annotations[0]
#             pprint(info_2d)
#             pprint(info_3d)
            try:
                object1_info_dict=info_2d["values"][0]
                object2_info_dict=info_2d["values"][1]
                object1_info=object1_info_dict["label_name"]
                object2_info=object2_info_dict["label_name"]
            #     pprint(object1_info)
            #     pprint(object2_info)
                object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
                object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]
            #     pprint(filename)
            #     pprint(object1_bb_2d)
            #     pprint(object2_bb_2d)
                object1_info_dict=info_3d["values"][0]
                object2_info_dict=info_3d["values"][1]
                object1_bb_3d=object1_info_dict["size"]
                object2_bb_3d=object2_info_dict["size"]
            except:
                object1_info=""
                object2_info=""
                object1_bb_2d=""
                object2_bb_2d=""
                object1_bb_3d=""
                object2_bb_3d=""

            new_row={
                "preposition":preposition_name,
                "category":category,
                "image_file_name":filename,
                "object_1_information":object1_info,
                "object_2_information":object2_info,
                "object_1_2d_bounding_boxes":object1_bb_2d,
                "object_2_2d_bounding_boxes":object2_bb_2d,
                "object_1_3d_bounding_boxes":object1_bb_3d,
                "object_2_3d_bounding_boxes":object2_bb_3d,    
            }
            df.loc[len(df)] = new_row
#         break
#     break

df

,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,above,small big person,rgb_2.png,small objects_lamp_model_normalized_18 Variant,person_person_4 Variant,"[202.0, 139.0, 7.0, 21.0]","[161.0, 254.0, 76.0, 185.0]","{'x': 0.548084438, 'y': 2.12284946, 'z': 1.421...","{'x': 11.6528873, 'y': 12.9343, 'z': 2.446444}"
1,above,small big person,rgb_3.png,small objects_pillow_model_normalized_13 Variant,person_person_6 Variant,"[163.0, 161.0, 22.0, 39.0]","[101.0, 352.0, 167.0, 160.0]","{'x': 2.90700722, 'y': 0.9630779, 'z': 1.91048...","{'x': 10.0679636, 'y': 9.553652, 'z': 2.09069061}"
2,above,small big person,rgb_4.png,small objects_clock_model_normalized_15 Variant,person_person_4 Variant,"[257.0, 143.0, 16.0, 28.0]","[186.0, 268.0, 155.0, 195.0]","{'x': 1.355826, 'y': 1.16213512, 'z': 0.4115899}","{'x': 11.1320543, 'y': 12.3561935, 'z': 2.3370..."
3,above,small big person,rgb_5.png,small objects_skateboard_model_normalized_16 V...,person_person_6 Variant,"[218.0, 47.0, 85.0, 76.0]","[214.0, 281.0, 96.0, 127.0]","{'x': 2.11818361, 'y': 1.650584, 'z': 6.424116}","{'x': 9.795555, 'y': 9.295161, 'z': 2.03412271}"
4,above,small big person,rgb_6.png,small objects_lamp_model_normalized_14 Variant,person_person_4 Variant,"[175.0, 155.0, 29.0, 23.0]","[104.0, 272.0, 159.0, 198.0]","{'x': 0.7989964, 'y': 2.225887, 'z': 0.798994064}","{'x': 11.3790627, 'y': 12.6303654, 'z': 2.388956}"
...,...,...,...,...,...,...,...,...,...
516,above,small big person,rgb_518.png,small objects_clock_model_normalized_23 Variant,person_person_5 Variant,"[216.0, 91.0, 18.0, 12.0]","[211.0, 242.0, 47.0, 134.0]","{'x': 1.3898809, 'y': 1.3898809, 'z': 0.09679857}","{'x': 9.707599, 'y': 10.3799458, 'z': 2.01750612}"
517,above,small big person,rgb_519.png,small objects_keyboard_model_normalized_30 Var...,person_person_4 Variant,"[273.0, 71.0, 21.0, 30.0]","[223.0, 237.0, 71.0, 202.0]","{'x': 2.59278345, 'y': 0.260665625, 'z': 1.057...","{'x': 10.6189871, 'y': 11.7867079, 'z': 2.229384}"
518,above,small big person,rgb_520.png,small objects_lamp_model_normalized_8 Variant,person_person_4 Variant,"[285.0, 108.0, 28.0, 31.0]","[275.0, 257.0, 171.0, 172.0]","{'x': 0.7165217, 'y': 2.18899179, 'z': 1.49153...","{'x': 10.1641579, 'y': 11.2818632, 'z': 2.1338..."
519,above,small big person,rgb_521.png,small objects_pillow_model_normalized_3 Variant,person_person_4 Variant,"[112.0, 12.0, 53.0, 54.0]","[150.0, 150.0, 70.0, 186.0]","{'x': 3.26477957, 'y': 1.13001609, 'z': 2.380878}","{'x': 11.7392712, 'y': 13.0301828, 'z': 2.4645..."


In [187]:
# test with 1 static example - 2 objects only - above
preposition_name="below"
root_folder_path="D:/SPUR NEURIPS"
preposition_folder_path=os.path.join(root_folder_path,preposition_name)

#create an empty df
columns=[
    "preposition",
    "category",
    "image_file_name",
    "object_1_information",
    "object_2_information",
    "object_1_2d_bounding_boxes",
    "object_2_2d_bounding_boxes",
    "object_1_3d_bounding_boxes",
    "object_2_3d_bounding_boxes",
]
 
df=pd.DataFrame(columns=columns)

categories=os.listdir(preposition_folder_path)
for category in tqdm(categories):
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    category_folder_path_list=sorted(category_folder_path_list)
    dataset_folder_path=os.path.join(category_folder_path,category_folder_path_list[0])
    dataset_folder_path_list=os.listdir(dataset_folder_path)
    captures_list=[x for x in dataset_folder_path_list if x.startswith("captures")]
    for captures_json_file in captures_list:
        full_json_path=os.path.join(dataset_folder_path,captures_json_file)
        with open(full_json_path) as file:
            json_data = json.load(file)
            for data in json_data["captures"]:
                filename=data["filename"].split("/")[-1]
                annotations=data["annotations"]
                if annotations[0]["id"]=="bounding box 3D":
                    info_3d=annotations[0]
                    info_2d=annotations[1]
                else:
                    info_3d=annotations[1]
                    info_2d=annotations[0]

                try:
                    object1_info_dict=info_2d["values"][0]
                    object2_info_dict=info_2d["values"][1]
                    object1_info=object1_info_dict["label_name"]
                    object2_info=object2_info_dict["label_name"]

                    object1_bb_2d=[object1_info_dict["x"],object1_info_dict["y"],object1_info_dict["width"],object1_info_dict["height"]]
                    object2_bb_2d=[object2_info_dict["x"],object2_info_dict["y"],object2_info_dict["width"],object2_info_dict["height"]]

                    object1_info_dict=info_3d["values"][0]
                    object2_info_dict=info_3d["values"][1]
                    object1_bb_3d=object1_info_dict["size"]
                    object2_bb_3d=object2_info_dict["size"]
                except:
                    object1_info=""
                    object2_info=""
                    object1_bb_2d=""
                    object2_bb_2d=""
                    object1_bb_3d=""
                    object2_bb_3d=""
                    print(f"error for {preposition_name}, {category}, {filename}, df index: {len(df)} ")

                new_row={
                    "preposition":preposition_name,
                    "category":category,
                    "image_file_name":filename,
                    "object_1_information":object1_info,
                    "object_2_information":object2_info,
                    "object_1_2d_bounding_boxes":object1_bb_2d,
                    "object_2_2d_bounding_boxes":object2_bb_2d,
                    "object_1_3d_bounding_boxes":object1_bb_3d,
                    "object_2_3d_bounding_boxes":object2_bb_3d,    
                }
                df.loc[len(df)] = new_row

df

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.39it/s]


,preposition,category,image_file_name,object_1_information,object_2_information,object_1_2d_bounding_boxes,object_2_2d_bounding_boxes,object_1_3d_bounding_boxes,object_2_3d_bounding_boxes
0,below,furniture furniture,rgb_2.png,furniture_microwave_model_normalized_28 Variant,furniture_microwave_model_normalized_9 Variant,"[153.0, 124.0, 64.0, 74.0]","[197.0, 335.0, 58.0, 60.0]","{'x': 4.71236, 'y': 2.69989514, 'z': 2.38217044}","{'x': 4.70499229, 'y': 2.09042645, 'z': 3.0277..."
1,below,furniture furniture,rgb_3.png,furniture_table_model_normalized_13 Variant,furniture_stove_model_normalized_18 Variant,"[73.0, 160.0, 170.0, 98.0]","[104.0, 430.0, 99.0, 52.0]","{'x': 8.897922, 'y': 3.86387515, 'z': 5.519822}","{'x': 4.16218758, 'y': 1.2486583, 'z': 4.404978}"
2,below,furniture furniture,rgb_4.png,furniture_microwave_model_normalized_11 Variant,furniture_bed_model_normalized_2 Variant,"[217.0, 127.0, 77.0, 57.0]","[150.0, 296.0, 263.0, 216.0]","{'x': 4.176357, 'y': 2.172134, 'z': 3.15226078}","{'x': 7.91541433, 'y': 10.5797644, 'z': 13.442..."
3,below,furniture furniture,rgb_5.png,furniture_stove_model_normalized_22 Variant,furniture_table_model_normalized_8 Variant,"[230.0, 74.0, 76.0, 38.0]","[186.0, 261.0, 145.0, 87.0]","{'x': 4.489478, 'y': 0.908768952, 'z': 3.9690752}","{'x': 10.3031015, 'y': 4.13222027, 'z': 4.4939..."
4,below,furniture furniture,rgb_6.png,furniture_microwave_model_normalized_9 Variant,furniture_microwave_model_normalized_5 Variant,"[150.0, 145.0, 79.0, 66.0]","[124.0, 361.0, 72.0, 53.0]","{'x': 4.56189966, 'y': 2.02685046, 'z': 2.9356...","{'x': 4.947596, 'y': 2.33335924, 'z': 1.622467}"
...,...,...,...,...,...,...,...,...,...
4995,below,vehicles vehicles,rgb_49.png,vehicles_motorcycle_model_normalized_18 Variant,vehicles_motorcycle_model_normalized_26 Variant,"[130.0, 73.0, 131.0, 96.0]","[210.0, 365.0, 46.0, 89.0]","{'x': 2.55518985, 'y': 7.20174551, 'z': 12.189...","{'x': 3.68977356, 'y': 4.10723257, 'z': 13.912..."
4996,below,vehicles vehicles,rgb_50.png,vehicles_bus_model_normalized_22 Variant,vehicles_bus_model_normalized_29 Variant,"[132.0, 108.0, 380.0, 127.0]","[41.0, 247.0, 431.0, 265.0]","{'x': 9.244875, 'y': 8.296465, 'z': 37.90234}","{'x': 10.6660748, 'y': 11.8382, 'z': 40.5224266}"
4997,below,vehicles vehicles,rgb_51.png,vehicles_bus_model_normalized_29 Variant,vehicles_bus_model_normalized_29 Variant,"[218.0, 0.0, 294.0, 194.0]","[123.0, 235.0, 389.0, 222.0]","{'x': 10.8080931, 'y': 11.9958258, 'z': 41.061...","{'x': 10.7983313, 'y': 11.9849939, 'z': 41.02489}"
4998,below,vehicles vehicles,rgb_52.png,vehicles_motorcycle_model_normalized_21 Variant,vehicles_motorcycle_model_normalized_9 Variant,"[200.0, 52.0, 99.0, 88.0]","[253.0, 278.0, 61.0, 46.0]","{'x': 3.41124964, 'y': 5.57235861, 'z': 11.447...","{'x': 4.42329741, 'y': 4.50555849, 'z': 12.064..."


In [185]:
df.iloc[3358]

preposition                                                               above
category                                                       small big person
image_file_name                                                     rgb_495.png
object_1_information          small objects_keyboard_model_normalized_27 Var...
object_2_information                                    person_person_3 Variant
object_1_2d_bounding_boxes                           [206.0, 113.0, 35.0, 16.0]
object_2_2d_bounding_boxes                         [154.0, 233.0, 128.0, 157.0]
object_1_3d_bounding_boxes    {'x': 2.63611841, 'y': 0.0466980748, 'z': 0.95...
object_2_3d_bounding_boxes    {'x': 12.2344027, 'y': 12.23549, 'z': 2.19381928}
Name: 3358, dtype: object

In [148]:
categories=os.listdir(preposition_folder_path)
for category in categories:
    category_folder_path=os.path.join(preposition_folder_path,category)
    category_folder_path_list=os.listdir(category_folder_path)
    rgb_path= os.path.join(category_folder_path,sorted(category_folder_path_list)[1])
    print(category,len(os.listdir(rgb_path)))

furniture furniture 469
furniture person 313
furniture vehicles 156
person furniture 313
person person 208
person small big 521
person vehicles 104
small big furniture 781
small big person 521
small big small big 651
small small small small 651
vehicles furniture 156
vehicles person 104
vehicles vehicles 52


In [ ]:
small big person